# Trigger Word Detection (TWD) Part 1

In [1]:
from pydub import AudioSegment as AS
from pydub.playback import play

import numpy as np
import random

import glob

E:\SiddC\ML_ANN\Anaconda\Anaconda\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
E:\SiddC\ML_ANN\Anaconda\Anaconda\lib\site-packages\pydub\utils.py:184: RuntimeWarning: Couldn't find ffplay or avplay - defaulting to ffplay, but may not work
  warn("Couldn't find ffplay or avplay - defaulting to ffplay, but may not work", RuntimeWarning)


In [2]:
file_dict = {}
i = 0

for filename in glob.glob('audio\*'):
    file_dict[i] = filename
    i = i + 1

In [3]:
flag = file_dict[12]
file_dict[12] = file_dict[29]
file_dict[29] = flag

In [4]:
def merge(bg, ls1, ls2):
    
    ls = ls1 + ls2
    max_gap = int((bg_time/( 2 * pos_neg_count)) * 10)
    ty = np.zeros((len(bg) * 16,),dtype = int)
    
    numbers = random.sample(range(0, 2 * pos_neg_count) , 2 * pos_neg_count)
    loc1 = len(bg) * random.randint(0, max_gap) / 100

    for i in range(len(numbers)):
        
        bg = bg.overlay(ls[numbers[i]], position = loc1)
        
        if numbers[i] < pos_neg_count:
            ty[int(loc1 + len(ls[numbers[i]])) * 16 : int(loc1 + len(ls[numbers[i]]) + 100) * 16] = 1 
            
        loc1 = (len(bg) * random.randint(10, max_gap) / 100) + loc1
        
        bg_array = bg.get_array_of_samples()
        
    return bg_array, ty, bg

In [5]:
tx_list = []
ty_list = []

pos_neg_count = 3
bg_time = 10 
db_fact = 15

for samp in range(100):
    pos = []
    neg = []
    
    rand = random.randint(7, db_fact)
    bg = (AS.from_wav(random.choice(glob.glob('audio\_background_noise_\*.wav')))
          [rand * 1000 : (rand + bg_time) * 1000]) - rand

    for i in range(pos_neg_count):
        filename = random.choice(glob.glob('audio\marvin\*.wav'))
        pos.append(AS.from_wav(filename))
            
    for j in range(pos_neg_count):
        choice = random.randint(0, 28)
        filename = random.choice(glob.glob(file_dict[choice] + '\*.wav'))
        neg.append(AS.from_wav(filename)) 
        
    tx, ty, wav_file = merge(bg, pos, neg)
    
    tx_list.append(tx)
    ty_list.append(ty)
    wav_file.export('generated\inp' + str(samp) + '.wav', format="wav")


In [6]:
trainy = np.stack(ty_list)
trainx = np.stack(tx_list)

np.save('generated\ trainx.npy', trainx)
np.save('generated\ trainy.npy', trainy)

print(trainx.shape,trainy.shape)

(100, 160000) (100, 160000)
